In [ ]:
# Imports and Initialization
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth

# Defining Spotify credentials (placeholders for actual values)
SPOTIFY_CLIENT_ID = ''
SPOTIFY_CLIENT_SECRET = ''
SPOTIFY_REDIRECT_URI = ''

# Initializing Spotify client with OAuth
sp_auth = SpotifyOAuth(client_id=SPOTIFY_CLIENT_ID, 
                       client_secret=SPOTIFY_CLIENT_SECRET, 
                       redirect_uri=SPOTIFY_REDIRECT_URI)
sp = spotipy.Spotify(auth_manager=sp_auth)


In [ ]:
# Playlist URI Extraction
def extract_playlist_uri(playlist_link):
    if 'open.spotify.com/playlist/' in playlist_link:
        return playlist_link.split('playlist/')[1].split('?')[0]
    return None

playlist_link = 'https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF'
playlist_URI = extract_playlist_uri(playlist_link)


In [ ]:
# Fetching Playlist Tracks with Enhanced Functionality
def get_playlist_tracks(sp_client, playlist_uri):
    results = sp_client.playlist_tracks(playlist_uri)
    tracks = results['items']
    while results['next']:
        results = sp_client.next(results)
        tracks.extend(results['items'])
    return tracks

tracks = get_playlist_tracks(sp, playlist_URI)

In [ ]:
# Extracting and Compiling Album, Artist, and Song Data
def compile_album_data(tracks):
    albums = []
    seen_albums = set()
    for track in tracks:
        album = track['track']['album']
        if album['id'] not in seen_albums:
            albums.append({
                'album_id': album['id'],
                'name': album['name'],
                'release_date': pd.to_datetime(album['release_date']),
                'total_tracks': album['total_tracks'],
                'url': album['external_urls']['spotify']
            })
            seen_albums.add(album['id'])
    return albums

def compile_artist_song_data(tracks):
    artists, songs = [], []
    seen_artists = set()
    for track in tracks:
        song_info = track['track']
        songs.append({
            'song_id': song_info['id'],
            'song_name': song_info['name'],
            'duration_ms': song_info['duration_ms'],
            'url': song_info['external_urls']['spotify'],
            'popularity': song_info['popularity'],
            'song_added': pd.to_datetime(track['added_at']),
            'album_id': song_info['album']['id'],
            'artist_id': song_info['album']['artists'][0]['id']
        })
        for artist in song_info['artists']:
            if artist['id'] not in seen_artists:
                artists.append({
                    'artist_id': artist['id'],
                    'artist_name': artist['name'],
                    'external_url': artist['href']
                })
                seen_artists.add(artist['id'])
    return artists, songs

album_data = compile_album_data(tracks)
album_df = pd.DataFrame(album_data)
artist_data, song_data = compile_artist_song_data(tracks)
artist_df = pd.DataFrame(artist_data)
song_df = pd.DataFrame(song_data)

In [ ]:
# Exporting Data to Excel (OPTIONAL STEP)
with pd.ExcelWriter('spotify_data.xlsx') as writer:
    album_df.to_excel(writer, sheet_name='Albums', index=False)
    artist_df.to_excel(writer, sheet_name='Artists', index=False)
    song_df.to_excel(writer, sheet_name='Songs', index=False)